# Segmentation of Image into foreground & background

In [50]:
import cv2
import numpy as np
import os

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf


In [3]:
IMG_WIDTH = 384
IMG_HEIGHT = 384
IMG_CHANNELS = 3

### Resized images keeping the Aspect ratio and padding with zeros

In [4]:
def resize(input_folder, output_folder, model_input_size):
    for file in os.listdir(input_folder):
        image = cv2.imread(os.path.join(input_folder,file))
        height, width = image.shape[:2]
        if model_input_size[0] / height < model_input_size[1] / width:
            scale = model_input_size[0] / height
        else:
            scale = model_input_size[1] / width
        new_width = int(scale * width)
        new_height = int(scale * height)
        resized_image = cv2.resize(image, (new_width, new_height))
        final_image = np.zeros(model_input_size)
        final_image[int((model_input_size[0] - new_height) / 2) : int((model_input_size[0] - new_height) / 2 + new_height), int((model_input_size[1] - new_width) / 2) : int((model_input_size[1] - new_width) / 2 + new_width), : ] = resized_image
        cv2.imwrite(os.path.join(output_folder, file), final_image)

In [5]:
gt_input = 'Original_Data/Ground_truth/gt'
gt_output = 'Resized_Data/Ground_truth/gt'

In [6]:
#resize(gt_input, gt_output, [IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])

In [7]:
melanoma_input = 'Original_Data/Melanoma/melanoma'
melanoma_output = 'Resized_Data/Melanoma/melanoma'

In [8]:
#resize(melanoma_input, melanoma_output, [IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])

In [9]:
others_input = 'Original_Data/Others/others'
others_output = 'Resized_Data/Others/others'

In [10]:
#resize(others_input, others_output, [IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])

### Function for calculation of the Mean_IOU metric.

The mean IOU metric is calculated by the mean intersection over union over different thresholds.

In [11]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

### Dice Coefficient metric and Dice Coefficient loss

Dice Coefficient is (2 * |AB|)/(|A| + |B|)  
Dice Coefficient loss is 1 - Dice Coefficient  
The loss is the sum of Binary crossentropy and Dice Coefficient loss.

In [51]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_coef_loss(y_true, y_pred)

### Unet Network to Segment Images

In [ ]:
inputs = Input([IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])
lambda_layer = Lambda(lambda x : x/255) (inputs)
c1 = Conv2D(8, (3, 3), activation = 'elu', padding = 'same') (lambda_layer)
c1 = Conv2D(8, (3, 3), activation = 'elu', padding = 'same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation = 'elu', padding = 'same') (p1)
c2 = Conv2D(16, (3, 3), activation = 'elu', padding = 'same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation = 'elu', padding = 'same') (p2)
c3 = Conv2D(32, (3, 3), activation = 'elu', padding = 'same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation = 'elu', padding = 'same') (p3)
c4 = Conv2D(64, (3, 3), activation = 'elu', padding = 'same') (c4)
p4 = MaxPooling2D((2, 2)) (c4)

c5 = Conv2D(128, (3, 3), activation = 'elu', padding = 'same') (p4)
c5 = Conv2D(128, (3, 3), activation = 'elu', padding = 'same') (c5)

u6 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(64, (3, 3), activation = 'elu', padding = 'same') (u6)
c6 = Conv2D(64, (3, 3), activation = 'elu', padding = 'same') (c6)

u7 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(32, (3, 3), activation = 'elu', padding = 'same') (u7)
c7 = Conv2D(32, (3, 3), activation = 'elu', padding = 'same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(16, (3, 3), activation = 'elu', padding = 'same') (u8)
c8 = Conv2D(16, (3, 3), activation = 'elu', padding = 'same') (c8)

u9 = Conv2DTranspose(8, (2, 2), strides = (2, 2), padding = 'same') (c8)
u9 = concatenate([u9, c1])
c9 = Conv2D(8, (3, 3), activation = 'elu', padding = 'same') (u9)
c9 = Conv2D(8, (3, 3), activation = 'elu', padding = 'same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs = [inputs], outputs = [outputs])

model.compile(optimizer = 'adam', loss = bce_dice_loss, metrics = [mean_iou, dice_coef, 'accuracy'])
model.summary()

### Read Data

In [14]:
x_train = []
y_train = []

In [15]:
for image in os.listdir(melanoma_output):
    x_train.append(cv2.imread(os.path.join(melanoma_output,image)))
    ground_image = image[:12] + '_segmentation.png'
    gt_image = cv2.imread(os.path.join(gt_output,ground_image),0)
    gt_image = cv2.threshold(gt_image,127,255,cv2.THRESH_BINARY)[1]
    y_train.append(gt_image)
    
for image in os.listdir(others_output):
    x_train.append(cv2.imread(os.path.join(others_output,image)))
    ground_image = image[:12] + '_segmentation.png'
    gt_image = cv2.imread(os.path.join(gt_output,ground_image),0)
    gt_image = cv2.threshold(gt_image,127,255,cv2.THRESH_BINARY)[1]
    y_train.append(gt_image)

In [24]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [17]:
y_train = y_train / 255.0
y_train = np.expand_dims(y_train,axis = -1)

### Training

In [40]:
finetune_model_path = 'Trained/segmentor_bce.h5'

In [26]:
callbacks = [ModelCheckpoint(finetune_model_path, monitor='val_loss', save_best_only=True, verbose=2)
            ]

In [53]:
results = model.fit(x_train, y_train, validation_split=0.1, batch_size=4, epochs=40, callbacks = callbacks)

Train on 1800 samples, validate on 200 samples
Epoch 1/40
1800/1800 [==============================] - 162s 90ms/step - loss: 0.8815 - mean_iou: 0.5144 - dice_coef: 0.4422 - acc: 0.8857 - val_loss: 0.8842 - val_mean_iou: 0.5895 - val_dice_coef: 0.5002 - val_acc: 0.8523

Epoch 00001: val_loss did not improve
Epoch 2/40
1800/1800 [==============================] - 157s 87ms/step - loss: 0.7399 - mean_iou: 0.6089 - dice_coef: 0.5465 - acc: 0.8991 - val_loss: 0.9179 - val_mean_iou: 0.6181 - val_dice_coef: 0.4745 - val_acc: 0.8674

Epoch 00002: val_loss did not improve
Epoch 3/40
1800/1800 [==============================] - 162s 90ms/step - loss: 0.6594 - mean_iou: 0.6275 - dice_coef: 0.6054 - acc: 0.9117 - val_loss: 0.7492 - val_mean_iou: 0.6387 - val_dice_coef: 0.5374 - val_acc: 0.9035

Epoch 00003: val_loss did not improve
Epoch 4/40
1800/1800 [==============================] - 158s 88ms/step - loss: 0.5231 - mean_iou: 0.6516 - dice_coef: 0.6978 - acc: 0.9320 - val_loss: 0.4258 - val_mea

KeyboardInterrupt: 

### Mean_IOU = 0.8023

### Dice Coefficient = 0.8529

### Accuracy = 0.9696